In [10]:
import re
import os
from collections import defaultdict
from collections import Counter
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
from pylab import *
from decimal import Decimal
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from decimal import Decimal
from scipy.misc import comb
import glob
import pickle
from operator import itemgetter 
import itertools
from sklearn import preprocessing

In [157]:
# def load_data(filename):
#     data = np.loadtxt(filename,dtype='string')
#     return data

In [2]:
def unfold_list(temp,x):
    for t in temp:
        if isinstance(t,list):
            unfold_list(t,x)
        else:
            x.append(t)
    return x

In [3]:
def tokenize(dirname,n):
    tokens = []
    files = glob.glob(dirname)
    
    if n:
        files_list = files[:n]
    else:
        files_list = files
    for f in files_list:
        data = open(f)
        temp = []
        for line in data:
            line = re.sub('\d+','',line)
            temp.append(re.findall(r'\w+|[!?#$]',line.lower()))
        
        temp = unfold_list(temp,[])        
        c = Counter(temp)
        for word in c.keys():
            total_word_count = sum(c.values())
            c[word] /= 1.*total_word_count
        tokens.append(c)
    
    return tokens        

In [8]:
def pickle_files(datastruct,filename,load,dump):
    if load:
        data = pickle.load(open(filename,'rb'))
        print 'end of loading file'
        return data
    if dump:
        pickle.dump(datastruct,open(filename,'wb'))
        print 'end of writing file'

In [39]:
def get_vocab_doc_freq(list_of_docs):
    print 'get_vocab_doc_freq'
    tokens = []
    for doc in list_of_docs:
        for token in doc.keys():
            if token not in tokens:
                tokens.append(token)
    
            
    doc_freq = defaultdict(int)
    for i,t in enumerate(tokens):
        for doc in list_of_docs:
            if t in doc.keys():
                doc_freq[t] += 1
     
    vocab = [feature for feature in doc_freq if doc_freq[feature] >= 5]
#     freq = [(feature,doc_freq[feature]) for feature in vocab]
    
#     print vocab
    if len(vocab) > 3000:
        return vocab[:3000]
    else:
        return vocab
            
                

In [45]:
spam_files = tokenize('spam/*',None)
pickle_files(spam_files,'spam_tokens',load=False,dump=True)

get_vocab_doc_freq


In [ ]:
ham_files = tokenize('ham/*',None)
pickle_files(ham_files,'ham_tokens',load=False,dump=True)

In [7]:
# vocab = get_vocab_doc_freq(spam_files+ham_files)
# pickle_files(vocab,'vocabulary',load=False,dump=True)

In [12]:
spam = pickle_files(None,'spam_tokens',load=True,dump=False)
ham = pickle_files(None,'ham_tokens',load=True,dump=False)

end of loading file
end of loading file


In [17]:
# print ham[1]

In [13]:
vocabulary = get_vocab_doc_freq(spam+ham)

get_vocab_doc_freq


In [15]:
pickle_files(vocabulary,'vocabulary',load=False,dump=True)

end of writing file


In [11]:
voc = pickle_files(None,'vocabulary',load=True,dump=False)


end of loading file


In [40]:
def save_likelihood(filename):
    print 'saving likelihood'
    files = spam + ham
#     vocab = pickle_files(None,'vocabulary',load=True,dump=False)
    X = create_vector(files,voc)
    Y = [0]*len(ham) + [1]*len(spam)
    labels = [0,1]
    mean = dict()
    sigma = dict()
    likelihood = []
    
    for l in labels:
        mean[l] = calc_mean(X,Y,l)
        sigma[l] = calc_std_dev(X,Y,l)
    
    for i,x in enumerate(X):
        likelihood.append((likelihood_function(x,mean[0],sigma[0],voc),likelihood_function(x,mean[1],sigma[1],voc)))
        
    pickle_files(likelihood,filename,load=False,dump=True)
    print 'end of pickling likelihood values'
            

In [ ]:
save_likelihood('likelihood')

saving likelihood
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
likelihood_function
li

In [29]:
def create_vector(files,vocab):
    X = []
    for f in files:
        temp = [0]*len(vocab)
        for i,feature in enumerate(vocab):
            if feature in f.keys():
                temp[i] = f[feature]
        X.append(temp)
    
    X = np.array(X)
    return X
        

In [30]:
def indicator(Y,class_val):
    l = list()
    for idx,y in enumerate(Y):
        if y == class_val:
            l.append(idx)
    return l

In [31]:
def compute_prior(Y,class_val):
    m = len(Y)
    indices = indicator(Y,class_val)
    return (1.*len(indices)/m)

In [32]:
def calc_mean(X,Y,label):
    indices = indicator(Y,label)
    x = X[indices]
    return np.mean(x,axis=0)

In [33]:
def calc_std_dev(X,Y,label):
    indices = indicator(Y,label)
    x = X[indices]
    return np.std(x,axis=0)

In [25]:
def likelihood_function(x,mean,sigma,vocab):
    print 'likelihood_function'
    p = 0.0
    for i,xi in enumerate(x):
        if xi in vocab:
            c = (1/(sigma[i]*math.sqrt(2*math.pi)))
            a = ((xi-mean[i])**2)/(2*(sigma[i]**2))
            p += math.log(c*math.exp(-a))
    
    return p


In [27]:
def predict_y(X,Y,labels,x_predict,prior,vocab):
    print 'predict_y'
    mean = {}
    sigma = {}
    y = []
   
    for l in labels:
        mean[l] = calc_mean(X,Y,l)
        sigma[l] = calc_std_dev(X,Y,l)
    

    for ind,x in enumerate(x_predict):
        temp = []
        for l in labels:
            a = (likelihood_function(x,mean[l],sigma[l],vocab))
            p = (a) + math.log(prior[l])
            temp.append(p)
        prob = 1.*temp[1]/sum(temp)
#         print prob
        
        if prob > 0.5:
            y.append(1)
        if prob < 0.5:
            y.append(0)
    
    return y

In [33]:
def cross_validation(dirname,k,verbose):

#     spam_files = tokenize('dirname/spam/*',None)
#     ham_files = tokenize('dirname/ham/*',None)
    
#     spam_tokens = pickle_files(None,'spam_tokens',load=True,dump=False)
#     ham_tokens = pickle_files(None,'ham_tokens',load=True,dump=False)
    files = spam + ham
    labels = [1]*len(spam) + [0]*len(ham)
    voc = pickle_files(None,'vocabulary',load=True,dump=False)
    
    classes = [0,1]
    
    
    accuracy = list()
    precision = list()
    recall = list()
    f_measure = list()
    prior = dict()
    fold = 1
    
    for train_ind, test_ind in KFold(len(files),k,shuffle=True,random_state=5):
         
        train_files = itemgetter(*train_ind)(files)
        test_files = itemgetter(*test_ind)(files)
        train_labels = itemgetter(*train_ind)(labels)
        test_labels = itemgetter(*test_ind)(labels)
        
#         vocab = get_vocab_doc_freq(train_files)
        
        X_train = create_vector(train_files,voc)
        Y_train = np.array(train_labels)
        X_test = create_vector(test_files,voc)
        Y_test = np.array(test_labels)
        
        for l in classes:
            prior[l] = compute_prior(Y_train,l)
            
       
        Y_predict = predict_y(X_train,Y_train,classes, X_test,prior,voc)
        
        
        acc = accuracy_score(Y_test,Y_predict,normalize=True, sample_weight=None)
        c_matrix = confusion_matrix(Y_test, Y_predict)
        prec = precision_score(Y_test, Y_predict) 
        rec = recall_score(Y_test, Y_predict)  
        fm = f1_score(Y_test, Y_predict)
    
        accuracy.append(acc)
        recall.append(rec)
        precision.append(prec)
        f_measure.append(fm)
        
        if verbose:
            print 'fold:', fold
            print 'accuracy:', acc
            print 'confusion_matrix'
            print c_matrix
            print 'prediction', prec
            print 'recall' , rec
            print 'f-measure',fm
            print ''
        fold += 1
    
    print accuracy
    print precision
    print recall
    print f_measure
    print ' '
    
    avg_acc = sum(accuracy)/len(accuracy)
    avg_pre = sum(precision)/len(precision)
    avg_rec = sum(recall)/len(recall)
    avg_fm = sum(f_measure)/len(f_measure)
   
    print 'avg_accuracy: ' , avg_acc
    print 'avg_precision', avg_pre
    print 'avg_recall', avg_rec
    print 'avg_fmeasure', avg_fm

In [68]:
cross_validation(k=10,verbose=True)

features 517
prior

/Library/Python/2.7/site-packages/ipykernel/__main__.py:14: RuntimeWarning: overflow encountered in double_scalars
/Library/Python/2.7/site-packages/ipykernel/__main__.py:14: RuntimeWarning: invalid value encountered in double_scalars


 {0: 0.5, 1: 0.5}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]

fold: 1
accuracy: 0.85
confusion_matrix
[[10  0]
 [ 3  7]]
prediction 1.0
recall 0.7
f-measure 0.823529411765

prior {0: 0.5, 1: 0.5}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]

fold: 2
accuracy: 0.9
confusion_matrix
[[12  0]
 [ 2  6]]
prediction 1.0
recall 0.75
f-measure 0.857142857143

prior {0: 0.5, 1: 0.5}
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1]

/Library/Python/2.7/site-packages/ipykernel/__main__.py:10: RuntimeWarning: divide by zero encountered in double_scalars
/Library/Python/2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: invalid value encountered in double_scalars
/Library/Python/2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: divide by zero encountered in double_scalars



[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1]

fold: 3
accuracy: 0.65
confusion_matrix
[[8 0]
 [7 5]]
prediction 1.0
recall 0.416666666667
f-measure 0.588235294118

prior {0: 0.5, 1: 0.5}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1]

fold: 4
accuracy: 0.55
confusion_matrix
[[11  0]
 [ 9  0]]
prediction 0.0
recall 0.0
f-measure 0.0

prior {0: 0.5, 1: 0.5}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1]
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1]

fold: 5
accuracy: 0.9
confusion_matrix
[[11  0]
 [ 2  7]]
prediction 1.0
recall 0.777777777778
f-measure 0.875

prior {0: 0.5, 1: 0.5}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1]

fold: 6
accuracy: 0.75
confusion_matrix
[[13  1]
 [ 4  2]]
prediction 0.666666666667
recall 0.333333333333
f-measure 0.444444444444

prior {0: 0.5, 1: 0.5}
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1

In [69]:
cross_validation(k=2,verbose=True)

features 517
prior

/Library/Python/2.7/site-packages/ipykernel/__main__.py:10: RuntimeWarning: divide by zero encountered in double_scalars
/Library/Python/2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: invalid value encountered in double_scalars
/Library/Python/2.7/site-packages/ipykernel/__main__.py:14: RuntimeWarning: overflow encountered in double_scalars
/Library/Python/2.7/site-packages/ipykernel/__main__.py:14: RuntimeWarning: invalid value encountered in double_scalars
/Library/Python/2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: divide by zero encountered in double_scalars


 {0: 0.5, 1: 0.5}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]

fold: 1
accuracy: 0.52
confusion_matrix
[[52  0]
 [48  0]]
prediction 0.0
recall 0.0
f-measure 0.0

prior {0: 0.5, 1: 0.5}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
cross_validation('data',k=10,verbose=True)